<a href="https://colab.research.google.com/github/dmnk1308/DubAir/blob/main/Colab_img_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import numpy as np
import pandas as pd
import cv2 
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
import itertools
import io
from sklearn import metrics
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt

# Helpers to turn any kind of data into bytes
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_png(value).numpy()])
    )

# WRITE

def serialize_example(image, label, path):
    '''
    Function that turns one set (!) of id, image, label into bytes

    Returns:    Representation of set in bytes
    '''  
    image = tf.io.decode_png(tf.io.read_file(path+image))
    feature = {
        #"id" : bytes_feature(id),
        'image' : image_feature(image),
        'label' : _int64_feature(label), 
    }

    #  Create a Features message using tf.train.Example (some tensorflow thing)
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def serialize_example_pred(image, path):
    '''
    Function that turns one set (!) of id, image, label into bytes

    Returns:    Representation of set in bytes
    '''  
    image = tf.io.decode_png(tf.io.read_file(path+image))
    feature = {
        #"id" : bytes_feature(id),
        'image' : image_feature(image),
    }

    #  Create a Features message using tf.train.Example (some tensorflow thing)
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def write_features(images, labels = None, path = "data/images_resized/", filename:str="images", prediction = False):
    '''
    Function that writes bytes into a tfrec-file
    '''  
    filename= filename+".tfr"
    writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our data to disk
    count = 0
    with tf.io.TFRecordWriter(filename) as writer:  
#        for id, image, label in zip(ids, images, labels):
      
      if prediction:
          for image in tqdm(images):
              try:
                  example = serialize_example_pred(image, path)
                  writer.write(example)
                  count += 1
              except:
                  continue
          return count

      for image, label in tqdm(zip(images, labels), total = len(labels)):
          try:
              example = serialize_example(image, label, path)
              writer.write(example)
              count += 1
          except:
            continue

    return count


# READ

def make_dataset(file_path, batch_size, seed = 123, prediction = False, training = False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """
  tf.random.set_seed(seed)
  def parse_example(example_proto, prediction):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth label.
    """

    if prediction:
        feature_spec = {
        #"id": tf.io.FixedLenFeature((), tf.string),
        'image': tf.io.FixedLenFeature((), tf.string),
        }

        example = tf.io.parse_single_example(example_proto, feature_spec)
    
        image = tf.io.decode_png(example['image'],channels =3)

        #image = tf.squeeze(image)
        example["input_1"] = image
        #id = example.pop("id") 
        example.pop('image') 

        return example


    feature_spec = {
        #"id": tf.io.FixedLenFeature((), tf.string),
        'image': tf.io.FixedLenFeature((), tf.string),
        'label': tf.io.FixedLenFeature((), tf.int64),
    }
    example = tf.io.parse_single_example(example_proto, feature_spec)
   # the image needs further treatment

    image = tf.io.decode_png(example['image'],channels =3)
    example["input_1"] = image
    label = example.pop('label')
    #id = example.pop("id") 
    example.pop('image') 

    return example, label

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
       dataset = dataset.shuffle(6000)
  dataset = dataset.map(lambda x: parse_example(x, prediction=prediction))
  dataset = dataset.batch(batch_size)
  return dataset

def show_category(cat, df, n = 10, folder = "data/images_resized/"):
    filter = df[cat] == 1
    path = folder + df["img_path"][filter][:n]
    l = df["label_raw"][filter][:n]
    id = df["img_path"][filter][:n]

    #path = random.choices(list(path), k=n)

    for p, lab, i in zip(path, l, id):
        img = plt.imread(str(p))
        print(i) 
        print(lab)
        plt.imshow(img)
        plt.show()



######################## MODEL ############################

def train_model(traindata, valdata, lr, epochs, logdir,  label_book, weights, callbacks = [], l2_reg = 0, do = 0):
   
    # Define Input
    inputs = keras.layers.Input(shape = (None, None, 3), name = "input_1")
    x = tf.cast(inputs, tf.float32)
    x = keras.applications.resnet50.preprocess_input(x)

    # Load ResNet with pretrained Imagenet weights
    resnet = keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg")

    # freeze the weights
    resnet.trainable = False
    x = resnet(x)

    # Layer 2 - Dense ReLu
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dropout(do)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(512, activation = "relu", kernel_regularizer = keras.regularizers.l2(l2_reg))(x)

    # Layer 3 - Dense ReLu 
    x = keras.layers.Dropout(do)(x)
    x = keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(7)(x)

    # Combine pretrained and output model
    model = keras.Model(inputs, outputs)

    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    callbacks.append(tensorboard_callback)
 
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"])

    history = model.fit(traindata, validation_data = valdata, epochs = epochs, callbacks = callbacks, class_weight = weights)
    return model, history

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_dataset = make_dataset("/content/drive/MyDrive/Dubair/train.tfr", batch_size = 32, training = True)
test_dataset = make_dataset("/content/drive/MyDrive/Dubair/test.tfr", batch_size = 32)
val_dataset = make_dataset("/content/drive/MyDrive/Dubair/val.tfr", batch_size = 32)

In [4]:
y_train = np.concatenate([y for x, y in train_dataset], axis=0)
#X_train = np.concatenate([x for x, y in train_dataset], axis=0)

In [5]:
label_cat = ["bathroom", "bedroom", "dining", "hallway", "kitchen", "living", "others"]
label = np.arange(7)
label_book = pd.DataFrame({"label": label_cat, "categorical_label": label})

In [6]:
weights = 1/(pd.Series(y_train).value_counts()/len(y_train))
weights = weights.to_dict()
weights

{0: 5.911514253980007,
 1: 3.462055507372073,
 2: 20.8718954248366,
 3: 28.411032028469748,
 4: 6.480113636363637,
 5: 9.240162037037038,
 6: 5.0931419457735245}

In [ ]:
lrs = [0.01, 0.001, 0.0001]
l2_reg = [0.001, 0.01, 0]
for lr in lrs:
    for l2 in l2_reg:
        logdir = "logs/labeling_cat7/resnet_lr_"+str(lr)+"_l2_"+str(l2)
        callbacks = []   
        model = train_model(train_dataset, val_dataset, epochs = 10, lr = lr, callbacks = callbacks, weights = weights, logdir = logdir, label_book = label_book, l2_reg = l2)
        model.save(logdir)

94781440/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
# some callbacks:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
logdir = "/content/drive/MyDrive/Dubair/logs/labeling_new/final_model_cat7_0102"

callbacks = [early_stopping, reduce_lr]
lr = 0.0001
final_model, history = train_model(train_dataset, val_dataset, epochs = 100, lr = lr, callbacks = callbacks, logdir = logdir, weights = weights, label_book = label_book, l2_reg = 0)
final_model.save(logdir)


94781440/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/100
499/499 [==============================] - 93s 158ms/step - loss: 7.8873 - accuracy: 0.6848 - val_loss: 0.6811 - val_accuracy: 0.7983 - lr: 1.0000e-04
Epoch 2/100
499/499 [==============================] - 76s 150ms/step - loss: 5.5133 - accuracy: 0.7757 - val_loss: 0.6257 - val_accuracy: 0.8192 - lr: 1.0000e-04
Epoch 3/100
499/499 [==============================] - 76s 150ms/step - loss: 4.9226 - accuracy: 0.7943 - val_loss: 0.5894 - val_accuracy: 0.8287 - lr: 1.0000e-04
Epoch 4/100
499/499 [==============================] - 79s 156ms/step - loss: 4.4639 - accuracy: 0.8160 - val_loss: 0.5800 - val_accuracy: 0.8361 - lr: 1.0000e-04
Epoch 5/100
499/499 [==============================] - 79s 156ms/step - loss: 4.0640 - accuracy: 0.8230 - val_loss: 0.5620 - val_accuracy: 0.8321 - lr: 1.0000e-04
Epoch 6/100
499/499 [==============================] - 79s 156ms/step - loss: 3.8187 - accuracy: 0.8295 - val_loss: 0.5447 - val_accuracy: 0.8439 - lr: 1.0000e-04
Epoch 7/100
499/499 [=

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
final_model.save_weights("weights.h5")

In [7]:
path_final_model = '/content/drive/MyDrive/Dubair/logs/labeling/final_model_cat7_0102'
final_model = tf.keras.models.load_model(path_final_model)

In [ ]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open("/content/drive/MyDrive/Dubair/logs/labeling_new", 'w'))

IsADirectoryError: ignored

In [8]:
final_model.evaluate(test_dataset)

62/62 [==============================] - 33s 290ms/step - loss: 0.4840 - accuracy: 0.8687


[0.4839659631252289, 0.8686612844467163]